In [15]:
import numpy as np
import pandas as pd

In [16]:
hdf = pd.HDFStore('/data/sushen/marinedebris/project/dataset.h5', mode = 'r')
df_train = hdf.select('train')
hdf.close()

In [17]:
df_train.sample(5)

,Class,Conf,B1,B2,B3,B4,B5,B6,B7,B8,...,NDWI,NRD,NDMI,BSI,con,dis,homo,ener,cor,asm
11,3.0,1.0,1116.0,824.0,511.0,237.0,181.0,150.0,126.0,108.0,...,0.651050,-129.0,0.728000,-0.571669,2.420272,1.253072,0.489242,0.243299,0.031256,0.059194
92,7.0,1.0,1246.0,934.0,625.0,342.0,292.0,264.0,259.0,212.0,...,0.493429,-130.0,0.399340,-0.451552,2.061031,1.090946,0.550084,0.238911,0.281039,0.057079
21,14.0,1.0,1294.0,1325.0,941.0,646.0,414.0,338.0,308.0,419.0,...,0.383824,-227.0,0.624031,-0.402493,3.146100,1.159989,0.577728,0.247583,0.453910,0.061298
225,10.0,1.0,1254.0,966.0,770.0,381.0,305.0,254.0,250.0,208.0,...,0.574642,-173.0,0.419795,-0.431707,1.088007,0.764290,0.650227,0.322111,0.410368,0.103756
31,6.0,1.0,1354.0,1017.0,716.0,448.0,397.0,361.0,331.0,253.0,...,0.477812,-195.0,0.213429,-0.349628,1.007212,0.712874,0.672997,0.272450,0.664378,0.074229


In [18]:
# Old Classes
# '0': 'Unresolved error during prediction pipeline',
# '1': 'Marine Debris',
# '2': 'Dense Sargassum',
# '3': 'Sparse Sargassum',
# '4': 'Natural Organic Material',
# '5': 'Ship',
# '6': 'Clouds',
# '7': 'Marine Water',
# '8': 'Sediment-Laden Water',
# '9': 'Foam',
# '10': 'Turbid Water',
# '11': 'Shallow Water',
# '12': 'Waves',
# '13': 'Cloud Shadows',
# '14': 'Wakes',
# '15': 'Mixed Water'


# New classes 
# 1. Marine debris: Marine debris (Old 1)
# 2. Sargassum: Sargassums together (Old 2, 3)
# 3. Natural organic material: Natural organic material (Old 4)
# 4. Water: All waters together + sediment water (Old 7, 8, 10, 11, 15)
# 5. Waves: Wave, foam, wakes together (Old 9, 12, 14)

# Removed
# Cloud and cloud shadows (Old 6, 13)
# Ship (Old 5)


In [24]:
new_cat_mapping = { '1': 'Marine Debris',
                '2': 'Sargassum',
                '3': 'Sargassum',
                '4': 'Natural Organic Material',
                '7': 'Water',
                '8': 'Water',
                '9': 'Waves',
                '10': 'Water',
                '11': 'Water',
                '12': 'Waves',
                '14': 'Waves',
                '15': 'Water'}

def new_cat_map(x):
    return new_cat_mapping[x]

new_cat_mapping_vec = np.vectorize(new_cat_map)

In [20]:
df_train['Class'].value_counts()

7.0     53334
6.0     36002
5.0     23913
1.0     23456
10.0    23223
14.0    15974
12.0    10482
3.0      7672
11.0     7660
13.0     6948
8.0      4363
2.0      3932
4.0      2790
15.0     2727
9.0      1118
Name: Class, dtype: int64

In [23]:
df_train = df_train[~df_train['Class'].isin([5.0, 6.0, 13.0])]
df_train['Class'].value_counts()

7.0     53334
1.0     23456
10.0    23223
14.0    15974
12.0    10482
3.0      7672
11.0     7660
8.0      4363
2.0      3932
4.0      2790
15.0     2727
9.0      1118
Name: Class, dtype: int64

In [25]:
df_train['Class'] = df_train['Class'].apply(lambda x: new_cat_mapping_vec(str(int(x))))

In [30]:
df_train.sample(5)

,Class,Conf,B1,B2,B3,B4,B5,B6,B7,B8,...,NDWI,NRD,NDMI,BSI,con,dis,homo,ener,cor,asm
198,Water,1.0,1297.0,980.0,703.0,390.0,319.0,259.0,244.0,189.0,...,0.576233,-201.0,0.453846,-0.434356,2.012954,1.091480,0.544871,0.220397,0.449287,0.048575
11,Water,1.0,1207.0,868.0,572.0,386.0,312.0,304.0,314.0,245.0,...,0.400245,-141.0,0.215881,-0.343392,2.597089,1.142762,0.552438,0.225774,0.671260,0.050974
36,Water,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
218,Water,1.0,1197.0,845.0,558.0,269.0,215.0,174.0,147.0,129.0,...,0.624454,-140.0,0.743243,-0.543582,2.491186,1.185363,0.535544,0.257951,0.041011,0.066539
22,Water,1.0,1403.0,1143.0,792.0,561.0,496.0,479.0,483.0,437.0,...,0.288853,-124.0,0.100756,-0.265012,2.652110,1.228766,0.521466,0.205358,0.404583,0.042172


In [27]:
df_train['Class'].value_counts()

Water                       91307
Waves                       27574
Marine Debris               23456
Sargassum                   11604
Natural Organic Material     2790
Name: Class, dtype: int64

In [31]:
new_dataset_path = '/data/sushen/marinedebris/project/dataset_new_classes.h5'
hdf_new = pd.HDFStore(new_dataset_path, mode = 'w')
hdf_new.append('train', df_train, format='table', data_columns=True)
hdf_new.close()

In [36]:
hdf_test = pd.HDFStore(new_dataset_path, mode = 'r')
df_trial = hdf_test.select('train')
hdf_test.close()
df_trial.sample(5)

,Class,Conf,B1,B2,B3,B4,B5,B6,B7,B8,...,NDWI,NRD,NDMI,BSI,con,dis,homo,ener,cor,asm
8,Marine Debris,3.0,1369.0,1248.0,941.0,658.0,486.0,406.0,427.0,608.0,...,0.214977,-50.0,0.74212,-0.425499,1.480769,0.896100,0.610417,0.253693,0.516349,0.064360
6,Waves,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
7,Sargassum,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
32,Marine Debris,1.0,1348.0,981.0,777.0,595.0,698.0,614.0,672.0,396.0,...,0.324808,-199.0,0.14121,-0.213216,5.961138,1.565839,0.517123,0.197309,0.559722,0.038931
24,Water,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
df_trial['Class'].value_counts()

Water                       91307
Waves                       27574
Marine Debris               23456
Sargassum                   11604
Natural Organic Material     2790
Name: Class, dtype: int64